## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [2]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.28 ms ± 19.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.5 ms ± 27.1 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.5 ms ± 50.9 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [4]:
#a)aceleración con CuPy

import cupy as cp
from cupyx.profiler import benchmark

# 1. Caso con transferencia de datos (CPU -> GPU -> Cálculo -> CPU)
def cupy_con_transferencia(x_cpu, y_cpu, a, b, c):
    # Mover a la GPU
    x_gpu = cp.asarray(x_cpu)
    y_gpu = cp.asarray(y_cpu)
    # Calcular
    z_gpu = a * x_gpu**2 + b * y_gpu + c
    # Devolver a la CPU
    return cp.asnumpy(z_gpu)

print("--- Rendimiento CuPy (incluyendo transferencia) ---")
print(benchmark(cupy_con_transferencia, (a_cpu, b_cpu, a, b, c), n_repeat=5))

# 2. Caso sin transferencia (Arrays creados directamente en la GPU)
a_gpu = cp.random.rand(size, dtype=cp.float64)
b_gpu = cp.random.rand(size, dtype=cp.float64)

def cupy_sin_transferencia(x_gpu, y_gpu, a, b, c):
    return a * x_gpu**2 + b * y_gpu + c

print("\n--- Rendimiento CuPy (sin transferencia) ---")
print(benchmark(cupy_sin_transferencia, (a_gpu, b_gpu, a, b, c), n_repeat=5))

--- Rendimiento CuPy (incluyendo transferencia) ---
cupy_con_transferencia:    CPU: 20366.628 us   +/- 248.582 (min: 20027.183 / max: 20617.817) us     GPU-0: 20374.061 us   +/- 248.214 (min: 20034.784 / max: 20624.737) us

--- Rendimiento CuPy (sin transferencia) ---
cupy_sin_transferencia:    CPU:   112.123 us   +/-  5.236 (min:   107.840 / max:   122.353) us     GPU-0:  4267.968 us   +/-  3.043 (min:  4265.664 / max:  4273.920) us


In [6]:
#b)aceleración con numba (GPU)

from numba import vectorize, cuda

# Definimos la ufunc para la GPU (target='cuda')
@vectorize(['float64(float64, float64, float64, float64, float64)'], target='cuda')
def grade2_numba_gpu(x, y, a, b, c):
    return a * x**2 + b * y + c

# 1. Caso con transferencia automática (Numba mueve los arrays por ti)
def numba_gpu_auto():
    return grade2_numba_gpu(a_cpu, b_cpu, a, b, c)

print("--- Rendimiento Numba GPU (con transferencia automática) ---")
print(benchmark(numba_gpu_auto, (), n_repeat=5))

# 2. Caso sin transferencia (Copiamos manualmente fuera de la medición)
a_gpu_numba = cuda.to_device(a_cpu)
b_gpu_numba = cuda.to_device(b_cpu)

def numba_gpu_directo():
    # Esta llamada es instantánea porque los datos ya están en la GPU
    return grade2_numba_gpu(a_gpu_numba, b_gpu_numba, a, b, c)

print("\n--- Rendimiento Numba GPU (sin transferencia) ---")
print(benchmark(numba_gpu_directo, (), n_repeat=5))

--- Rendimiento Numba GPU (con transferencia automática) ---
numba_gpu_auto      :    CPU: 17766.937 us   +/- 2064.366 (min: 16039.725 / max: 20770.960) us     GPU-0: 17776.212 us   +/- 2065.019 (min: 16047.615 / max: 20781.153) us

--- Rendimiento Numba GPU (sin transferencia) ---
numba_gpu_directo   :    CPU:  2000.910 us   +/- 1257.344 (min:   854.682 / max:  3546.122) us     GPU-0:  2256.064 us   +/- 1056.139 (min:  1271.808 / max:  3553.120) us


c) Análisis de resultados: CuPy vs Numba (GPU)

Tras ejecutar las pruebas con un vector de 5.000.000 de elementos, los tiempos obtenidos muestran lo siguiente:

Método	Con Transferencia	Sin Transferencia (Cálculo puro)
CuPy	~20.37 ms	4.27 ms
Numba GPU	~18.85 ms	2.34 ms
CPU (Numba njit)	---	8.44 ms
CPU (NumPy)	---	18.40 ms

2. Conclusiones principales

    El coste de la transferencia (PCIe): Se observa que mover los datos de la CPU a la GPU y viceversa consume aproximadamente 16 ms (la diferencia entre los tiempos con y sin transferencia). En este ejercicio, la transferencia tarda mucho más que el propio cálculo. Esto nos enseña que para vectores "pequeños" (como 5 millones), a veces es más rápido usar Numba en CPU (8.44 ms) que mover todo a la GPU y traerlo de vuelta (18.85 ms).

    Potencia de cálculo de la GPU: Si miramos solo el tiempo de ejecución en la tarjeta (sin copia), la GPU es masivamente superior. Numba GPU (2.34 ms) es casi 4 veces más rápido que el Numba en CPU (8.44 ms) y 8 veces más rápido que el NumPy estándar.

    Numba vs CuPy: En este caso específico, Numba GPU ha sido más eficiente que CuPy (2.34 ms vs 4.27 ms). Esto suele ocurrir porque Numba genera un "kernel" único que calcula toda la expresión de una vez, mientras que CuPy a veces tiene que lanzar varias operaciones pequeñas consecutivas.

    Eficiencia de memoria: La reducción de tiempo de ~18 ms (NumPy) a ~2.3 ms (Numba GPU) demuestra que las GPUs son ideales para operaciones vectoriales ("Data Parallel") donde se puede procesar cada elemento de forma independiente.